# Display of the performance results for PGW and the PW initialization

In this notebook we run the code implemented in 'partial_gw.py' which is adapted from Chapel et al.

We compute PGW in the setting of domain adaptation with the same of different features, and with a prior set to either 10 or 20%. We also compute the initialization Partial-Wasserstein when the feature types are the same.

In [1]:
import os
import utils
import numpy as np
from IPython.display import HTML, display, Markdown
import pandas as pd
from sklearn.metrics import accuracy_score, recall_score, precision_score

import ot
import partial_gw as pgw
import matplotlib.pyplot as plt

name_path = 'results_pgw'
path = os.getcwd() + "/saved_plans"
if not os.path.isdir(path):
    os.mkdir(path)
path = path + "/" + name_path
if not os.path.isdir(path):
    os.mkdir(path)

# Caltech dataset - PU on different domains
## Prior is set to 10%

In [7]:
n_unl = 100
n_pos = 100
nb_reps = 20
nb_dummies = 10
prior = 0.1
domain_s = ['surf_Caltech', 'surf_amazon', 'surf_webcam', 'surf_dslr']
domain_d = ['decaf_caltech', 'decaf_amazon', 'decaf_webcam', 'decaf_dslr']

In [6]:
domain_s = ['surf_Caltech', 'surf_amazon', 'surf_webcam', 'surf_dslr']
for d in domain_s:
    pgw.compute_perf_pgw('surf_Caltech', d, n_pos, n_unl, prior, nb_reps, 'results_pgw', nb_dummies)
    
domain_d = ['decaf_caltech', 'decaf_amazon', 'decaf_webcam', 'decaf_dslr']
for d in domain_d:
    pgw.compute_perf_pgw('decaf_caltech', d, n_pos, n_unl, prior, nb_reps, 'results_pgw', nb_dummies)

In [8]:
domain_s = ['surf_Caltech', 'surf_amazon', 'surf_webcam', 'surf_dslr']
domain_d = ['decaf_caltech', 'decaf_amazon', 'decaf_webcam', 'decaf_dslr']
np.set_printoptions(precision=5)
datasets = [('surf_Caltech', d) for d in domain_s] + [('decaf_caltech', d) for d in domain_d]
for (d_p, d_u) in datasets:
    acc, rec, prec = [], [], []
    
    # Aggregate and compute accuracies
    for i in range(nb_reps):
        P, U, y_u = utils.draw_p_u_dataset_scar(d_p, d_u, n_pos, n_unl, prior, i)
        nb_unl_pos = int(np.sum(y_u))
        
        plan = np.load(path + f'/partial_gw_plan_{d_p}_{n_pos}_{d_u}_{n_unl}_prior{prior}_reps{i}.npy', allow_pickle=True)
        marg = np.sum(plan[-nb_dummies:], axis=0)
        
        y_hat = np.ones(len(y_u))
        y_hat[np.argsort(marg)[nb_unl_pos:]] = 0
        
        acc.append(accuracy_score(y_u, y_hat))
        rec.append(recall_score(y_u, y_hat))
        prec.append(precision_score(y_u, y_hat))
    
    # Compute mean
    print(f'Accuracy for dataset {(d_p, d_u, prior)}: {100 * np.mean(acc)} $\pm$ {100 * np.std(acc)}')
#     print(f'Recall for dataset {(d_p, d_u)}: {np.mean(rec)} $\pm$ {np.std(rec)}')
#     print(f'Precision for dataset {(d_p, d_u)}: {np.mean(prec)} $\pm$ {np.std(prec)}\n')

Accuracy for dataset ('surf_Caltech', 'surf_Caltech', 0.1): 84.89999999999999 $\pm$ 1.9467922333931804
Accuracy for dataset ('surf_Caltech', 'surf_amazon', 0.1): 82.2 $\pm$ 1.8867962264113187
Accuracy for dataset ('surf_Caltech', 'surf_webcam', 0.1): 81.30000000000001 $\pm$ 1.8193405398660225
Accuracy for dataset ('surf_Caltech', 'surf_dslr', 0.1): 81.4 $\pm$ 1.6852299546352687
Accuracy for dataset ('decaf_caltech', 'decaf_caltech', 0.1): 83.0 $\pm$ 2.2360679774997885
Accuracy for dataset ('decaf_caltech', 'decaf_amazon', 0.1): 81.4 $\pm$ 1.9078784028338882
Accuracy for dataset ('decaf_caltech', 'decaf_webcam', 0.1): 82.70000000000002 $\pm$ 2.5514701644346127
Accuracy for dataset ('decaf_caltech', 'decaf_dslr', 0.1): 83.79999999999998 $\pm$ 1.5362291495737233


## Prior is set to 20%

In [12]:
n_unl = 100
n_pos = 100
nb_reps = 20
nb_dummies = 10
prior = 0.2
domain_s = ['surf_Caltech', 'surf_amazon', 'surf_webcam']
domain_d = ['decaf_caltech', 'decaf_amazon', 'decaf_webcam']

In [9]:
n_unl = 100
n_pos = 100
nb_reps = 20
nb_dummies = 10
prior = 0.2

domain_s = ['surf_Caltech', 'surf_amazon', 'surf_webcam']
for d in domain_s:
    pgw.compute_perf_pgw('surf_Caltech', d, n_pos, n_unl, prior, nb_reps, 'results_pgw', nb_dummies)
    
domain_d = ['decaf_caltech', 'decaf_amazon', 'decaf_webcam']
for d in domain_d:
    pgw.compute_perf_pgw('decaf_caltech', d, n_pos, n_unl, prior, nb_reps, 'results_pgw', nb_dummies)

In [13]:
np.set_printoptions(precision=5)
datasets = [('surf_Caltech', d) for d in domain_s] + [('decaf_caltech', d) for d in domain_d]
for (d_p, d_u) in datasets:
    acc, rec, prec = [], [], []
    
    # Aggregate and compute accuracies
    for i in range(nb_reps):
        P, U, y_u = utils.draw_p_u_dataset_scar(d_p, d_u, n_pos, n_unl, prior, i)
        nb_unl_pos = int(np.sum(y_u))
        
        plan = np.load(path + f'/partial_gw_plan_{d_p}_{n_pos}_{d_u}_{n_unl}_prior{prior}_reps{i}.npy', allow_pickle=True)
        marg = np.sum(plan[-nb_dummies:], axis=0)
        
        y_hat = np.ones(len(y_u))
        y_hat[np.argsort(marg)[nb_unl_pos:]] = 0
        
        acc.append(accuracy_score(y_u, y_hat))
        rec.append(recall_score(y_u, y_hat))
        prec.append(precision_score(y_u, y_hat))
    
    # Compute mean
    print(f'Accuracy for dataset {(d_p, d_u, prior)}: {100 * np.mean(acc)} $\pm$ {100 * np.std(acc)}')
#     print(f'Recall for dataset {(d_p, d_u)}: {np.mean(rec)} $\pm$ {np.std(rec)}')
#     print(f'Precision for dataset {(d_p, d_u)}: {np.mean(prec)} $\pm$ {np.std(prec)}\n')

Accuracy for dataset ('surf_Caltech', 'surf_Caltech', 0.2): 75.7 $\pm$ 4.659399102888697
Accuracy for dataset ('surf_Caltech', 'surf_amazon', 0.2): 65.99999999999999 $\pm$ 3.286335345030996
Accuracy for dataset ('surf_Caltech', 'surf_webcam', 0.2): 64.30000000000001 $\pm$ 2.123676058159532
Accuracy for dataset ('decaf_caltech', 'decaf_caltech', 0.2): 76.70000000000002 $\pm$ 3.593048844644337
Accuracy for dataset ('decaf_caltech', 'decaf_amazon', 0.2): 68.7 $\pm$ 4.394314508543966
Accuracy for dataset ('decaf_caltech', 'decaf_webcam', 0.2): 75.9 $\pm$ 3.6041642581880216


# Caltech dataset - PU on different feature spaces
## Prior is set tio 10%

In [14]:
n_unl = 100
n_pos = 100
nb_reps = 20
nb_dummies = 10
prior = 0.1
domain_s = ['surf_Caltech', 'surf_amazon', 'surf_webcam', 'surf_dslr']
domain_d = ['decaf_caltech', 'decaf_amazon', 'decaf_webcam', 'decaf_dslr']

In [11]:
n_unl = 100
n_pos = 100
nb_reps = 20
nb_dummies = 10
prior = 0.1

domain_s = ['surf_Caltech', 'surf_amazon', 'surf_webcam', 'surf_dslr']
for d in domain_s:
    pgw.compute_perf_pgw('decaf_caltech', d, n_pos, n_unl, prior, nb_reps, 'results_pgw', nb_dummies)
    
domain_d = ['decaf_caltech', 'decaf_amazon', 'decaf_webcam', 'decaf_dslr']
for d in domain_d:
    pgw.compute_perf_pgw('surf_Caltech', d, n_pos, n_unl, prior, nb_reps, 'results_pgw', nb_dummies)

In [15]:
np.set_printoptions(precision=5)
datasets = [('surf_Caltech', d) for d in domain_d] + [('decaf_caltech', d) for d in domain_s]
for (d_p, d_u) in datasets:
    acc, rec, prec = [], [], []
    
    # Aggregate and compute accuracies
    for i in range(nb_reps):
        P, U, y_u = utils.draw_p_u_dataset_scar(d_p, d_u, n_pos, n_unl, prior, i)
        nb_unl_pos = int(np.sum(y_u))
        
        plan = np.load(path + f'/partial_gw_plan_{d_p}_{n_pos}_{d_u}_{n_unl}_prior{prior}_reps{i}.npy', allow_pickle=True)
        marg = np.sum(plan[-nb_dummies:], axis=0)
        
        y_hat = np.ones(len(y_u))
        y_hat[np.argsort(marg)[nb_unl_pos:]] = 0
        
        acc.append(accuracy_score(y_u, y_hat))
        rec.append(recall_score(y_u, y_hat))
        prec.append(precision_score(y_u, y_hat))
    
    # Compute mean
    print(f'Accuracy for dataset {(d_p, d_u, prior)}: {100 * np.mean(acc)} $\pm$ {100 * np.std(acc)}')
#     print(f'Recall for dataset {(d_p, d_u)}: {np.mean(rec)} $\pm$ {np.std(rec)}')
#     print(f'Precision for dataset {(d_p, d_u)}: {np.mean(prec)} $\pm$ {np.std(prec)}\n')

Accuracy for dataset ('surf_Caltech', 'decaf_caltech', 0.1): 85.1 $\pm$ 2.643860813280458
Accuracy for dataset ('surf_Caltech', 'decaf_amazon', 0.1): 87.10000000000001 $\pm$ 6.49538297562199
Accuracy for dataset ('surf_Caltech', 'decaf_webcam', 0.1): 88.6 $\pm$ 5.624944444170091
Accuracy for dataset ('surf_Caltech', 'decaf_dslr', 0.1): 91.10000000000001 $\pm$ 8.086408349817608
Accuracy for dataset ('decaf_caltech', 'surf_Caltech', 0.1): 81.00000000000001 $\pm$ 1.6124515496597067
Accuracy for dataset ('decaf_caltech', 'surf_amazon', 0.1): 81.2 $\pm$ 1.9390719429665286
Accuracy for dataset ('decaf_caltech', 'surf_webcam', 0.1): 81.30000000000001 $\pm$ 2.0273134932713264
Accuracy for dataset ('decaf_caltech', 'surf_dslr', 0.1): 80.80000000000001 $\pm$ 1.599999999999997


## Prior is set to 20%

In [16]:
n_unl = 100
n_pos = 100
nb_reps = 20
nb_dummies = 10
prior = 0.2
domain_s = ['surf_Caltech', 'surf_amazon', 'surf_webcam']
domain_d = ['decaf_caltech', 'decaf_amazon', 'decaf_webcam']

In [14]:
n_unl = 100
n_pos = 100
nb_reps = 20
nb_dummies = 10
prior = 0.2

domain_s = ['surf_Caltech', 'surf_amazon', 'surf_webcam']
for d in domain_s:
    pgw.compute_perf_pgw('decaf_caltech', d, n_pos, n_unl, prior, nb_reps, 'results_pgw', nb_dummies)
    
domain_d = ['decaf_caltech', 'decaf_amazon', 'decaf_webcam']
for d in domain_d:
    pgw.compute_perf_pgw('surf_Caltech', d, n_pos, n_unl, prior, nb_reps, 'results_pgw', nb_dummies)

In [17]:
np.set_printoptions(precision=5)
datasets = [('surf_Caltech', d) for d in domain_d] + [('decaf_caltech', d) for d in domain_s]
for (d_p, d_u) in datasets:
    acc, rec, prec = [], [], []
    
    # Aggregate and compute accuracies
    for i in range(nb_reps):
        P, U, y_u = utils.draw_p_u_dataset_scar(d_p, d_u, n_pos, n_unl, prior, i)
        nb_unl_pos = int(np.sum(y_u))
        
        plan = np.load(path + f'/partial_gw_plan_{d_p}_{n_pos}_{d_u}_{n_unl}_prior{prior}_reps{i}.npy', allow_pickle=True)
        marg = np.sum(plan[-nb_dummies:], axis=0)
        
        y_hat = np.ones(len(y_u))
        y_hat[np.argsort(marg)[nb_unl_pos:]] = 0
        
        acc.append(accuracy_score(y_u, y_hat))
        rec.append(recall_score(y_u, y_hat))
        prec.append(precision_score(y_u, y_hat))
    
    # Compute mean
    print(f'Accuracy for dataset {(d_p, d_u, prior)}: {100 * np.mean(acc)} $\pm$ {100 * np.std(acc)}')
#     print(f'Recall for dataset {(d_p, d_u)}: {np.mean(rec)} $\pm$ {np.std(rec)}')
#     print(f'Precision for dataset {(d_p, d_u)}: {np.mean(prec)} $\pm$ {np.std(prec)}\n')

Accuracy for dataset ('surf_Caltech', 'decaf_caltech', 0.2): 75.6 $\pm$ 3.322649545167231
Accuracy for dataset ('surf_Caltech', 'decaf_amazon', 0.2): 87.89999999999999 $\pm$ 6.617401302626283
Accuracy for dataset ('surf_Caltech', 'decaf_webcam', 0.2): 88.4 $\pm$ 5.817215828899594
Accuracy for dataset ('decaf_caltech', 'surf_Caltech', 0.2): 63.69999999999999 $\pm$ 2.917190429162966
Accuracy for dataset ('decaf_caltech', 'surf_amazon', 0.2): 62.39999999999999 $\pm$ 4.454211490264018
Accuracy for dataset ('decaf_caltech', 'surf_webcam', 0.2): 61.4 $\pm$ 3.231098884280704


# Compute performance of initialization - Partial Wasserstein
We focus in this section on the computation of Partial-Wasserstein (PW), to check if PGW improves the performance.

We only focus on this section on PU learning performed over similar features.

## Prior is set to 10%

In [18]:
n_unl = 100
n_pos = 100
nb_reps = 20
nb_dummies = 10
prior = 0.1
domain_s = ['surf_Caltech', 'surf_amazon', 'surf_webcam', 'surf_dslr']
domain_d = ['decaf_caltech', 'decaf_amazon', 'decaf_webcam', 'decaf_dslr']

In [ ]:
domain_s = ['surf_Caltech', 'surf_amazon', 'surf_webcam', 'surf_dslr']
for d in domain_s:
    pgw.compute_perf_init('surf_Caltech', d, n_pos, n_unl, prior, nb_reps, 'results_pgw', nb_dummies)
    
domain_d = ['decaf_caltech', 'decaf_amazon', 'decaf_webcam', 'decaf_dslr']
for d in domain_d:
    pgw.compute_perf_init('decaf_caltech', d, n_pos, n_unl, prior, nb_reps, 'results_pgw', nb_dummies)

In [19]:
datasets = [('surf_Caltech', d) for d in domain_s] + [('decaf_caltech', d) for d in domain_d]
for (d_p, d_u) in datasets:
    acc, rec, prec = [], [], []
    
    # Aggregate and compute accuracies
    for i in range(nb_reps):
        P, U, y_u = utils.draw_p_u_dataset_scar(d_p, d_u, n_pos, n_unl, prior, i)
        nb_unl_pos = int(np.sum(y_u))
        
        plan = np.load(path + f'/partial_gw_init_{d_p}_{n_pos}_{d_u}_{n_unl}_prior{prior}_reps{i}.npy', allow_pickle=True)
        marg = np.sum(plan[-nb_dummies:], axis=0)
        
        y_hat = np.ones(len(y_u))
        y_hat[np.argsort(marg)[nb_unl_pos:]] = 0
        
        acc.append(accuracy_score(y_u, y_hat))
        rec.append(recall_score(y_u, y_hat))
        prec.append(precision_score(y_u, y_hat))
    
    # Compute mean
    print(f'Accuracy for dataset {(d_p, d_u, prior)}: {100 * np.mean(acc)} $\pm$ {100 * np.std(acc)}')
#     print(f'Recall for dataset {(d_p, d_u)}: {np.mean(rec)} $\pm$ {np.std(rec)}')
#     print(f'Precision for dataset {(d_p, d_u)}: {np.mean(prec)} $\pm$ {np.std(prec)}\n')

Accuracy for dataset ('surf_Caltech', 'surf_Caltech', 0.1): 89.9 $\pm$ 2.046948949045873
Accuracy for dataset ('surf_Caltech', 'surf_amazon', 0.1): 81.80000000000001 $\pm$ 1.2489995996796768
Accuracy for dataset ('surf_Caltech', 'surf_webcam', 0.1): 81.89999999999999 $\pm$ 1.3379088160259627
Accuracy for dataset ('surf_Caltech', 'surf_dslr', 0.1): 80.00000000000001 $\pm$ 1.1102230246251565e-14
Accuracy for dataset ('decaf_caltech', 'decaf_caltech', 0.1): 93.9 $\pm$ 1.6093476939431057
Accuracy for dataset ('decaf_caltech', 'decaf_amazon', 0.1): 80.10000000000002 $\pm$ 0.43588989435406533
Accuracy for dataset ('decaf_caltech', 'decaf_webcam', 0.1): 80.10000000000002 $\pm$ 0.43588989435406533
Accuracy for dataset ('decaf_caltech', 'decaf_dslr', 0.1): 80.60000000000002 $\pm$ 0.9165151389911639


## Prior is set to 20%

In [20]:
n_unl = 100
n_pos = 100
nb_reps = 20
nb_dummies = 10
prior = 0.2
domain_s = ['surf_Caltech', 'surf_amazon', 'surf_webcam']
domain_d = ['decaf_caltech', 'decaf_amazon', 'decaf_webcam']

In [ ]:
domain_s = ['surf_Caltech', 'surf_amazon', 'surf_webcam']
for d in domain_s:
    pgw.compute_perf_init('surf_Caltech', d, n_pos, n_unl, prior, nb_reps, 'results_pgw', nb_dummies)
    
domain_d = ['decaf_caltech', 'decaf_amazon', 'decaf_webcam']
for d in domain_d:
    pgw.compute_perf_init('decaf_caltech', d, n_pos, n_unl, prior, nb_reps, 'results_pgw', nb_dummies)

In [21]:
datasets = [('surf_Caltech', d) for d in domain_s] + [('decaf_caltech', d) for d in domain_d]
for (d_p, d_u) in datasets:
    acc, rec, prec = [], [], []
    
    # Aggregate and compute accuracies
    for i in range(nb_reps):
        P, U, y_u = utils.draw_p_u_dataset_scar(d_p, d_u, n_pos, n_unl, prior, i)
        nb_unl_pos = int(np.sum(y_u))
        
        plan = np.load(path + f'/partial_gw_init_{d_p}_{n_pos}_{d_u}_{n_unl}_prior{prior}_reps{i}.npy', allow_pickle=True)
        marg = np.sum(plan[-nb_dummies:], axis=0)
        
        y_hat = np.ones(len(y_u))
        y_hat[np.argsort(marg)[nb_unl_pos:]] = 0
        
        acc.append(accuracy_score(y_u, y_hat))
        rec.append(recall_score(y_u, y_hat))
        prec.append(precision_score(y_u, y_hat))
    
    # Compute mean
    print(f'Accuracy for dataset {(d_p, d_u, prior)}: {100 * np.mean(acc)} $\pm$ {100 * np.std(acc)}')
#     print(f'Recall for dataset {(d_p, d_u)}: {np.mean(rec)} $\pm$ {np.std(rec)}')
#     print(f'Precision for dataset {(d_p, d_u)}: {np.mean(prec)} $\pm$ {np.std(prec)}\n')

Accuracy for dataset ('surf_Caltech', 'surf_Caltech', 0.2): 79.7 $\pm$ 2.9849623113198587
Accuracy for dataset ('surf_Caltech', 'surf_amazon', 0.2): 65.6 $\pm$ 2.870540018881464
Accuracy for dataset ('surf_Caltech', 'surf_webcam', 0.2): 65.10000000000001 $\pm$ 1.946792233393179
Accuracy for dataset ('decaf_caltech', 'decaf_caltech', 0.2): 90.60000000000002 $\pm$ 2.5377155080899043
Accuracy for dataset ('decaf_caltech', 'decaf_amazon', 0.2): 62.499999999999986 $\pm$ 2.085665361461421
Accuracy for dataset ('decaf_caltech', 'decaf_webcam', 0.2): 65.7 $\pm$ 1.926136028425824
